In [ ]:
#important librairies
import numpy as np
import cv2
from google.colab.patches import cv2_imshow


In [ ]:
# video path

from google.colab import drive
drive.mount('/content/drive')
video_path = '/content/drive/MyDrive/TASK1 VIDEO HUMANOID - Made with Clipchamp_1717567695472.mp4'
cap = cv2.VideoCapture(video_path)
output_path = 'output_video.mp4' # path where I will store the output video

In [ ]:
# EARLIER i was thinking to skip frames as it could lead to lesser data and more easy processing , but it eventually leads to loss of data and efficiency , thus I dropped the idea

fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

centers = [] # array of centers

In [ ]:
#checking the fps
print("Frames per second:", fps)

In [ ]:
#function to detect the center of the archery , I tried to sense the yellow colour and then

def detect_yellow_disc_center(frame):
  hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)#converting rgb to hsv

    # # Define the range for yellow color in HSV
  lower_yellow = np.array([20, 100, 100])
  upper_yellow = np.array([30, 255, 255])

    # # Create a mask for yellow color
  mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # # Apply the mask to get the yellow regions
  yellow_regions = cv2.bitwise_and(frame, frame, mask=mask)

    # # Convert the yellow regions to grayscale
  gray_yellow = cv2.cvtColor(yellow_regions, cv2.COLOR_BGR2GRAY)

    # # Apply GaussianBlur to the grayscale yellow regions
  blurred = cv2.GaussianBlur(gray_yellow, (5, 5), 0)

    # # Apply Canny edge detection
  edges = cv2.Canny(blurred, 100, 200)

    # # Detect circles using Hough Circle Transform
  circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50,
                         param1=50, param2=30, minRadius=5, maxRadius=100)

  if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            return (x, y), r

  return None, None

  cv2.imshow("output", image)
  cv2.waitKey(0)

In [ ]:
def calculate_velocity(centers, fps):
    velocities = []
    for i in range(1, len(centers)):
        if centers[i] is not None and centers[i-1] is not None:
            dx = centers[i][0] - centers[i-1][0]
            dy = centers[i][1] - centers[i-1][1]
            distance = np.sqrt(dx**2 + dy**2)
            velocity = (dx * fps, dy * fps)
            velocities.append(velocity)
    return velocities

In [ ]:
def predict_next_center(current_center, velocity, fps,scale=1.5):
    dx, dy = velocity
    next_center = (int(current_center[0] + scale*dx / fps), int(current_center[1] +scale* dy / fps))
    return next_center

In [ ]:
centers = []
velocities = []

frame_index = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    center, radius = detect_yellow_disc_center(frame)
    if center is not None and len(centers) > 0:
        previous_center = centers[-1]
        velocity = calculate_velocity([previous_center, center], fps)[0]
        velocities.append(velocity)
        predicted_center = predict_next_center(previous_center, velocity, fps)

        # Draw actual center and circle
        cv2.circle(frame, center, radius, (0, 255, 0), 2)  # Actual circle in green
        cv2.circle(frame, center, 5, (0, 0, 255), -1)  # Actual center in red

        # Draw predicted center
        cv2.circle(frame, predicted_center,radius, (255, 0, 0), 2)  # Predicted center in blue
        cv2.circle(frame, predicted_center, 5, (0, 0, 255), -1)
        print(f"Predicted Center = {predicted_center}")
    else:
        predicted_center = None

    centers.append(center)

    # Draw the trajectory of the center
    for i in range(1, len(centers)):
        if centers[i] is not None and centers[i-1] is not None:
            cv2.line(frame, centers[i-1], centers[i], (255, 0, 0), 2)  # Draw lines in blue

    out.write(frame)
    cv2_imshow(frame)  # Use cv2_imshow instead of cv2.imshow for Google Colab

    frame_info = f"Frame {frame_index}: Actual Center = {center}, Predicted Center = {predicted_center}, Radius = {radius}"
    print(frame_info)

    frame_index += 1

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
if velocities:
    avg_velocity = sum(np.linalg.norm(v) for v in velocities) / len(velocities)
    print(f"Average Velocity = {avg_velocity:.2f} pixels/second")
else:
    print("No velocities calculated")

# Print individual frame velocities
for i, velocity in enumerate(velocities):
    print(f"Frame {i + 1}: Velocity = {np.linalg.norm(velocity):.2f} pixels/second")

In [ ]:
from google.colab import files
files.download(output_path)